In [1]:
print("--------------- Empezando ---------------")

--------------- Empezando ---------------


In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
from pytz import timezone
from datetime import timedelta
import requests
import datetime
import numpy as np 
import os
import time
import json
from google_cloud import BigQuery
import base64

import boto3
s3 = boto3.client('s3')
s3.download_file("fury-data-apps", "marketing-utils/pzenone/utils.py","utils.py")
import utils

AUTH_BIGQUERY = base64.b64decode(os.environ["SECRET_AUTH_BIGQUERY_MODEL"])


aux = json.loads(AUTH_BIGQUERY)
filename = 'CREDENTIAL.json'
with open(filename, 'w') as f:
    json.dump(aux, f)

In [29]:
def BQ_insert_time(time, SITE, BU, INICIATIVA, JOB, PHOTO_ID):
    """
    Inserta el START de un proceso
    """
    # Instancio BigQuery
    bqte = BigQuery(AUTH_BIGQUERY)

    # For INSERT
    Q = """
        INSERT `meli-marketing.MODELLING.JOBS_MONITOR` (SITE, BU, INICIATIVA, JOB,R_START_DATE, PHOTO_ID)
        VALUES('"""+SITE+"""', '"""+BU+"""', '"""+INICIATIVA+"""', '"""+JOB+"""', TIMESTAMP('"""+time+"""'),DATE('"""+PHOTO_ID+"""'))
        """
    bqte.execute(Q)        


def BQ_update_time(mode, time, SITE, BU, INICIATIVA, JOB, PHOTO_ID):
    """
    Updatea el START o END time del proceso
    mode: START/END
    time: i.e 2021-04-07 08:00:00
    """

    # Instancio BigQuery
    bqte = BigQuery(AUTH_BIGQUERY)

    Q = """
            UPDATE `meli-marketing.MODELLING.JOBS_MONITOR`
            SET R_"""+mode.upper()+"""_DATE = TIMESTAMP('"""+time+"""')
            WHERE SITE = '"""+SITE+"""'
            AND INICIATIVA = '"""+INICIATIVA+"""'
            AND BU = '"""+BU+"""'
            AND JOB = '"""+JOB+"""'
            AND PHOTO_ID = DATE('"""+PHOTO_ID+"""')
            """

    BQ = bqte.execute(Q)        



def send_alert(subject,body,mail_to):
    data = {"from": "pedro.zenone@mercadolibre.com",
               "to": mail_to,
               "subject": subject,
               "body": subject
              }
    url = "http://internal.mercadolibre.com/bi/send_mail"
    r = requests.post(url = url, json = data)

        
def get_tabla(PHOTO_ID = '*'):
    """
    Obtiene la tabla completa o para un PHOTO_ID especifico
    """

    # Instancio BigQuery
    bqte = BigQuery(AUTH_BIGQUERY)
    
    if PHOTO_ID == '*':
        Q = """
            SELECT *
            FROM `meli-marketing.MODELLING.JOBS_MONITOR` 
            """
        df = bqte.execute_response(Q)
    else:
        Q = f"""
            SELECT *
            FROM `meli-marketing.MODELLING.JOBS_MONITOR` 
            WHERE PHOTO_ID = DATE('{PHOTO_ID}')
            """
        df = bqte.execute_response(Q)
        
    def my_tz_caster(col):
        try:
            x = pd.to_datetime(col).dt.tz_convert(None).dt.to_pydatetime()
        except:
            x = pd.Timestamp(None,tz=None)
        return x
            
    df["E_START_DATE"] = my_tz_caster(df.E_START_DATE)
    df["E_END_DATE"] = my_tz_caster(df.E_END_DATE)
    df["R_START_DATE"] = my_tz_caster(df.R_START_DATE)
    df["R_END_DATE"] = my_tz_caster(df.R_END_DATE)
    
    
    return df

def get_mail(SITE, BU, INICIATIVA, JOB):
    # Obtenemos la tabla de Google SpreadSheet

    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('CREDENTIAL.json', scope) # Your json file here
    gc = gspread.authorize(credentials)
    wks = gc.open("JOBS_MONITOR").sheet1
    data = wks.get_all_values()
    headers = data.pop(0)
    # Tabla con horarios
    df = pd.DataFrame(data, columns=headers)
    mail_to = df.loc[(df.SITE == SITE) & (df.INICIATIVA == INICIATIVA) & (df.BU == BU) & (df.SITE == SITE) & (df.JOB == JOB), 'MAIL'].values[0]
    return mail_to

In [3]:
def get_horarios():
    # Obtenemos la tabla de Google SpreadSheet

    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    credentials = ServiceAccountCredentials.from_json_keyfile_name('CREDENTIAL.json', scope) # Your json file here
    gc = gspread.authorize(credentials)
    wks = gc.open("JOBS_MONITOR").sheet1
    data = wks.get_all_values()
    headers = data.pop(0)
    # Tabla con horarios
    df = pd.DataFrame(data, columns=headers)
    # Calculo la fecha del lunes
    today = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires')).replace(tzinfo=None).date()
    lunes = today - timedelta(days = today.weekday())
    # Horario ESTIMADO de arranque
    # Fechas de cada uno de los jobs 
    week_days = {'LU':0,'MA':1, 'MI':2, 'JU':3, 'VI':4, 'SA':5, 'DO':6}
    df['DIA_SEMANA'] = df['DIA_SEMANA'].replace(week_days)

    df = df.loc[(df.DIA_SEMANA == today.weekday()) | (df.DIA_SEMANA == '')]

    df["E_START_DATE"] = pd.to_datetime(today)
    df["E_START_DATE"] = pd.to_datetime(today)
    df["E_END_DATE"] = pd.to_datetime(today)
    df["E_END_DATE"] = pd.to_datetime(today)
    # Pasamos timedelta los minutos y horas
    df["START_MINUTOS"] = df["START_MINUTOS"].apply(lambda x: timedelta(minutes=int(x)))
    df["START_HORA"] = df["START_HORA"].apply(lambda x: timedelta(hours=int(x)))
    df["END_MINUTOS"] = df["END_MINUTOS"].apply(lambda x: timedelta(minutes=int(x)))
    df["END_HORA"] = df["END_HORA"].apply(lambda x: timedelta(hours=int(x)))
    # Sumamos minutos y horas a la fecha 
    df["E_START_DATE"] = df["E_START_DATE"] + df["START_MINUTOS"] + df["START_HORA"]
    df["E_END_DATE"] = df["E_END_DATE"] + df["END_MINUTOS"] + df["END_HORA"]
    # Agregamos FECHA a definir y PHOTO ID
    df['R_END_DATE'] = np.nan
    df['R_START_DATE'] = np.nan
    df['PHOTO_ID'] = df["E_START_DATE"].apply(lambda x: x.date())
    df["ID"] = df.SITE+'_'+df.BU+'_'+df.INICIATIVA+'_'+df.JOB
    # Sort values
    df.sort_values(by="E_START_DATE", inplace=True)

    return df

In [4]:
def LOADER(tabla):
    # Chequeamos si ya existen registros en la tabla (por si fallo el LOADER y lo insertaron en la tabla los JOBS)
    # Saco de la tabla si existen procesos registrados con el PHOTO_ID de hoy

    print("--------------- Inicio LOADER ---------------")

    # Instancio BigQuery
    bqte = BigQuery(AUTH_BIGQUERY)

    # Inserto tabla con datos de hoy
    schedule = get_horarios()[["SITE", "BU", "INICIATIVA", "JOB", "E_START_DATE", "E_END_DATE", "R_END_DATE", "R_START_DATE", "PHOTO_ID"]]

    # Borro todo lo que no haya corrido aun del schedule que se haya cargado
    bqte.execute(f"""
        DELETE FROM `meli-marketing.MODELLING_STAGE.JOBS_MONITOR_STAGE` WHERE 1=1;
        """)

    for ind,line in schedule.iterrows():
        # INSERT
        Q2 = f"""
            INSERT `meli-marketing.MODELLING_STAGE.JOBS_MONITOR_STAGE` (SITE, BU, INICIATIVA, JOB, E_START_DATE, E_END_DATE, R_END_DATE, R_START_DATE, PHOTO_ID)
            VALUES('{line["SITE"]}','{line["BU"]}','{line["INICIATIVA"]}','{line["JOB"]}',TIMESTAMP('{line["E_START_DATE"]}'),TIMESTAMP('{line["E_END_DATE"]}'),
                    NULL,NULL,CURRENT_DATE('America/Argentina/Buenos_Aires'))
            """

        BQ_2 = bqte.execute(Q2)


    bqte.execute(f"""
        -- Limpio para no duplicar con stagining
        DELETE FROM `meli-marketing.MODELLING.JOBS_MONITOR` 
        WHERE PHOTO_ID = CURRENT_DATE('America/Argentina/Buenos_Aires')
                AND R_START_DATE is null AND R_END_DATE is null;

        DELETE FROM `meli-marketing.MODELLING.JOBS_MONITOR` 
        WHERE PHOTO_ID = CURRENT_DATE('America/Argentina/Buenos_Aires')
                AND SITE = 'LOADER';

        -- Inserto lo que no haya terminado
        INSERT INTO `meli-marketing.MODELLING.JOBS_MONITOR` (SITE, BU, INICIATIVA, JOB, E_START_DATE, E_END_DATE, R_END_DATE, R_START_DATE, PHOTO_ID)
        SELECT SITE, BU, INICIATIVA, JOB, E_START_DATE, E_END_DATE, R_END_DATE, R_START_DATE, PHOTO_ID
        FROM `meli-marketing.MODELLING_STAGE.JOBS_MONITOR_STAGE` s
        WHERE NOT EXISTS(SELECT 1 FROM `meli-marketing.MODELLING.JOBS_MONITOR` m 
                         WHERE m.SITE = s.SITE AND m.BU = s.BU AND m.INICIATIVA = s.INICIATIVA
                             AND m.JOB = s.JOB AND m.PHOTO_ID = s.PHOTO_ID);

        -- Vuelvo a cargar el LOADER
        INSERT `meli-marketing.MODELLING.JOBS_MONITOR` (SITE, BU, INICIATIVA, JOB, E_START_DATE, E_END_DATE, R_END_DATE, R_START_DATE, PHOTO_ID)
        VALUES('LOADER',NULL,NULL,NULL,NULL,NULL,NULL,CAST(CURRENT_DATETIME('America/Argentina/Buenos_Aires') AS TIMESTAMP),CURRENT_DATE('America/Argentina/Buenos_Aires'));    
        """)  

In [5]:
def CHECKER():
    print("--------------- Inicio CHECKER ---------------")

    # Horarios a comparar 
    today = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires'))
    fin = datetime.datetime(year=today.year, month=today.month, day=today.day, hour=23)

    # Instancio BigQuery
    bqte = BigQuery(AUTH_BIGQUERY)

    CHECKER = True

    # Excluidos condición 1 y 2
    exclude1 = []
    exclude2 = []

    today = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires'))

    # Esta comparación no anda bien
    while CHECKER == True:

        # Me levanto me traigo la lista de jobs

        A = get_tabla(str(today.date()))
        levanto = A["E_START_DATE"].dropna().append(A["E_END_DATE"].dropna()).drop_duplicates()
        today = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires'))
        levanto = levanto[levanto > pd.to_datetime(today.replace(tzinfo=None))].sort_values().reset_index(drop=True)

        if(levanto.shape[0] == 0):
            break  # listo no hay nada para revisar

        # Chequeo si paso la hora de E_START_DATE de algun job y no tiene R_START_DATE
        B = A.loc[(A.E_START_DATE <= pd.to_datetime(today.replace(tzinfo=None))) & (A.R_START_DATE.isnull())]

        # Si A contiene alguna observación
        if not B.empty:
            for row in B.iterrows():
                PROCESO = row[1].SITE+'_'+row[1].BU+'_'+row[1].INICIATIVA+'_'+row[1].JOB
                if (PROCESO in exclude1) == False:
                    subject = f'Falla en {PROCESO} | START'
                    body = f'El proceso {PROCESO} no inició en el horario que debería haber empezado.'
                    # mandar mail con nombre de job que fallo 
                    mail = get_mail(row[1].SITE, row[1].BU, row[1].INICIATIVA, row[1].JOB)
                    send_alert(subject, body, mail)
                    print('ALERTA 1 {PROCESO}')
                    exclude1.append(PROCESO)
                    print(exclude1)



        # Chequeo si paso la hora estimada de finalización de algún proceso y aún no finalizó
        B = A.loc[(A.E_END_DATE <= pd.to_datetime(today.replace(tzinfo=None))) & (A.R_END_DATE.isnull())]
        # Si contiene alguna observación
        if not B.empty:
            for row in B.iterrows():
                PROCESO = row[1].SITE+'_'+row[1].BU+'_'+row[1].INICIATIVA+'_'+row[1].JOB
                if (PROCESO in exclude2) == False:
                    subject = f'Falla en {PROCESO} | END'
                    body = f'El proceso {PROCESO} no finalizó en el horario que debería haber terminado.'
                    # mandar mail con nombre de job que fallo 
                    mail = get_mail(row[1].SITE, row[1].BU, row[1].INICIATIVA, row[1].JOB)
                    send_alert(subject, body, mail)
                    print(f'ALERTA 2 {PROCESO}')
                    exclude2.append(PROCESO)
                    print(exclude2)


        today = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires'))

        # Chequeamos si today < 23:00hs
        today = today.replace(tzinfo=None)
        fin = datetime.datetime(year=today.year, month=today.month, day=today.day, hour=23)
        CHECKER = today < fin

        dif = levanto[0] - today.replace(tzinfo=None)
        secs = int(dif.total_seconds())

        if(secs<=0):
            print("coninue")
            continue # por temas de timming

        # Mando a dormir hasta que arranque el nuevo check
        print("Sleeping:",secs)
        time.sleep(secs+1)

In [6]:
# DEBUG = os.environ["SECRET_DEBUG"]
# print("DEBUG:", DEBUG)
DEBUG = "False"

In [32]:
if DEBUG == "False":    
    # Inserta inicio JOB de LOADER en la tabla
    now = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires')).strftime('%Y-%m-%d %H:%M:%S')
    BQ_insert_time(now,'LOADER', '', '', '', now[0:10])
    time.sleep(5)
    try:
        LOADER(get_horarios())
    except Exception as inst:
        subject = 'Falla en LOADER'
        body = 'Fallo la función de LOADER'
        send_alert(subject, body, 'pedro.zenone@mercadolibre.com')    
        print(type(inst))    
        print(inst.args)    
        print(inst)         

--------------- Inicio LOADER ---------------


In [ ]:
if DEBUG == "False":
    try:
        CHECKER()
    except Exception as inst:
        subject = 'Falla en CHECKER'
        body = 'Fallo la función de CHECKER'
        send_alert(subject, body, 'pedro.zenone@mercadolibre.com')
        print(type(inst))    
        print(inst.args)    
        print(inst)         
    
    # Inserta final JOB de LOADER en tabla 
    now = datetime.datetime.now(timezone('America/Argentina/Buenos_Aires')).strftime('%Y-%m-%d %H:%M:%S')
    BQ_update_time('end', now, 'LOADER', '', '', '',  now[0:10])